In [1]:
#/!/usr/bin/python

# Imports:
from PIL import Image
import io
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import os
import random
import requests
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers
import zipfile

In [2]:
# change loglevel to avoid unnecessary tf errors
os.environ['TF_CPP_MIN_LEVEL'] = '2'

In [3]:
# Inlezen, filteren en splitten van data


data_dir = 'data'


train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir, validation_split=0.2, subset='training', seed=123, image_size=(224, 224), batch_size=32
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir, validation_split=0.2, subset='validation', seed=123, image_size=(224, 224), batch_size=32
)



Found 819 files belonging to 3 classes.
Using 656 files for training.
Found 819 files belonging to 3 classes.
Using 163 files for validation.


2022-10-31 19:46:41.128002: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
def inspectData():
    
    plt.figure(figsize=(10,10))
    
    for i in range(9):
        plt.subplot(3, 3, i+1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        number = i
        plt.imshow(train_ds[number])
        plt.xlabel(labelNames[trainLabels[number]]) 
    
    plt.show()
    
#inspectData()  

In [5]:
# Preprocess:

tf.keras.layers.Rescaling(scale=1./255, offset=0.0)

In [6]:
model = keras.models.Sequential()
model.add(layers.Input(shape=(224, 224, 3)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dropout(0.1))
model.add(layers.Dense(256, activation = tf.keras.layers.LeakyReLU(alpha=0.3)))
model.add(layers.Dense(4, activation='tanh'))
model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 128)     3584      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 128)    0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      73792     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 64)        36928     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 26, 26, 64)       0

In [7]:
model.compile(loss = 'sparse_categorical_crossentropy',  
   optimizer = 'Adam', metrics = 'accuracy')

model.fit(train_ds)

21/21 [==============================] - 38s 2s/step - loss: 1.8720 - accuracy: 0.1631
